In [15]:
from dustpy import Simulation
from dustpy import constants as c
from setup_ext_pe import param_ext_pe_FRIED, setup_ext_pe_FRIED
from functions_ext_pe import S_gas_epe, S_dust_epe

import numpy as np

In [22]:
sim = Simulation()
sim.ini.star.M = 0.3*c.M_sun
sim.ini.gas.Mdisk = 0.01*c.M_sun        
sim.ini.gas.SigmaRc = 50.*c.au          
sim.ini.gas.gamma = 1.0  
sim.ini.gas.alpha = 1e-3

# Relevant Dust parameters
sim.ini.dust.d2gRatio = 0.01                # Initial dust-to-gas ratio              
sim.ini.dust.vfrag = 1000.0                 # Dust fragmentation velocity [cm/s]

sim.ini.grid.Nr = 260
sim.ini.grid.rmin = 5*c.au
sim.ini.grid.rmax = 1000*c.au
sim.ini.grid.Nmbpd = 7
sim.ini.grid.mmin = 1.e-12
sim.ini.grid.mmax = 1.e+5

ext_pe=True
option_sqrt_grid = True         
option_track_lostdust = False
option_gasonly = True

#| to speed up the simulation in the gas-only case:
if option_gasonly:
    sim.ini.grid.Nmbpd = 2
    sim.ini.grid.mmax = 1e-10

#| refine the grid
if option_sqrt_grid:
    sim.grid.ri = np.square(np.linspace(np.sqrt(sim.ini.grid.rmin), np.sqrt(sim.ini.grid.rmax), num = sim.ini.grid.Nr +1))

#| initialize ext_pe parameters

sim.initialize()
param_ext_pe_FRIED(sim, UV_Flux=100.)

setup_ext_pe_FRIED(sim)

#| update the external source term
if ext_pe:
    sim.gas.S.ext.updater = S_gas_epe
    sim.dust.S.ext.updater = S_dust_epe
    sim.update()

if option_track_lostdust:
        setup_lostdust(sim, using_FRIED = True)


def setup_gasonly(sim):
    '''
    This routine deactivates all dust evolution source terms and integration instructions.
    Call it after setting up everything else in the simulation.
    '''
    sim.dust.S.coag[...] = 0.
    sim.dust.S.coag.updater = None
    sim.dust.S.ext[...] = 0.
    sim.dust.S.ext.updater = None
    sim.dust.S.hyd[...] = 0.
    sim.dust.S.hyd.updater = None
    sim.dust.S.tot[...] = 0.
    sim.dust.S.tot.updater = None
    sim.dust.S.updater = None
    del(sim.integrator.instructions[0])
    sim.update()


if option_gasonly:
    setup_gasonly(sim)

In [ ]:
snapshots1 = np.logspace(3,6,9,endpoint=False)
snapshots2 = np.linspace(1, 10, 19) * 1e+6

sim.t.snapshots = np.concatenate((snapshots1, snapshots2))* c.year

sim.writer.datadir='test_dir'
sim.writer.dumping = False
sim.writer.overwrite = True
sim.verbosity = 2
    
sim.run()